# Common Libraries

In [1]:
import folium
import numpy as np
import pandas as pd
from folium import RegularPolygonMarker
import math
from datetime import datetime

# Constants

In [2]:
loc_map_focus = [37.565898, 127.055460]

test_number = 7

# Paths

In [3]:
location_data_path = f"/Users/seojin/Desktop/Locs/location_data{test_number}.csv"
event_location_data_path = f"/Users/seojin/Desktop/Locs/event_location_data{test_number}.csv"
coordinate_data_path = f"/Users/seojin/Desktop/Locs/coordinate{test_number}.csv"

# Load geographic data

In [4]:
location_df = pd.read_csv(location_data_path)
location_df = location_df[location_df["EventType"] == "Receive"]
location_df["Time"] = [datetime.fromisoformat(time.replace('Z', '+00:00')) for time in location_df["Time"]]

In [5]:
event_df = pd.read_csv(event_location_data_path)
event_df["Time"] = [datetime.fromisoformat(time.replace('Z', '+00:00')) for time in event_df["Time"]]
initTime = event_df[event_df["EventType"] == "ReferenceCoord"]["Time"].iloc[0]
startTime = initTime - pd.Timedelta(seconds=20)

In [6]:
location_df["seconds_fromStart"] = [e.total_seconds() for e in location_df["Time"] - startTime]
location_df["seconds_fromInit"] = [e.total_seconds() for e in location_df["Time"] - initTime]

In [7]:
coordinate_df = pd.read_csv(coordinate_data_path)
coordinate_df["Time"] = [datetime.fromisoformat(time.replace('Z', '+00:00')) for time in coordinate_df["Time"]]

In [8]:
stard_10 = location_df[(location_df["seconds_fromStart"] > 0) & (location_df["seconds_fromStart"] < 10)]
stard_10[["Latitude", "Longitude"]].mean().to_numpy()

array([ 37.56589958, 127.05544897])

In [9]:
end_10 = location_df[(location_df["seconds_fromInit"] < 0) & (location_df["seconds_fromInit"] > -10)]
end_10[["Latitude", "Longitude"]].mean().to_numpy()

array([ 37.5659579 , 127.05531265])

# Map

In [10]:
start = event_df[event_df["EventType"] == "startCoord"].iloc[-1]
start_loc = start[["Latitude", "Longitude"]].to_numpy().astype(np.float64)

reference = event_df[event_df["EventType"] == "ReferenceCoord"].iloc[-1]
reference_loc = reference[["Latitude", "Longitude"]].to_numpy().astype(np.float64)

In [11]:
start_loc = [37.565777, 127.055805]
reference_loc = [37.565938, 127.055577]

In [12]:
# 지도 생성 (서울 좌표 기준)
m = folium.Map(location = loc_map_focus, zoom_start=18)

# folium.PolyLine(locations=line_points, color='blue', weight=5, opacity=0.8).add_to(m)
folium.Marker(start_loc, popup = "Start", icon=folium.Icon(color="red")).add_to(m)
folium.Marker(reference_loc, popup = "Ref", icon=folium.Icon(color="blue")).add_to(m)
folium.Marker([37.565898, 127.055460], popup = "Object", icon=folium.Icon(color="pink")).add_to(m)

# folium.RegularPolygonMarker(location = line_points[0], fill_color = 'blue', number_of_sides = 3, radius = 10, rotation = 100).add_to(m)
m

# Raw locations

In [13]:
m = folium.Map(location = loc_map_focus, zoom_start=18)
for lat, long in zip(stard_10["Latitude"], stard_10["Longitude"]):
    folium.Marker([lat, long], popup = "Start", icon=folium.Icon(color="red")).add_to(m)
m

In [14]:
m = folium.Map(location = loc_map_focus, zoom_start=18)
for lat, long in zip(end_10["Latitude"], end_10["Longitude"]):
    folium.Marker([lat, long], popup = "Ref", icon=folium.Icon(color="blue")).add_to(m)
m

In [15]:
m = folium.Map(location = loc_map_focus, zoom_start=18)
folium.Marker([37.565705, 127.055311], popup = "Start", icon=folium.Icon(color="red")).add_to(m)
folium.Marker([37.565938, 127.055577], popup = "End", icon=folium.Icon(color="blue")).add_to(m)
folium.Marker([37.565898, 127.055460], popup = "Obj", icon=folium.Icon(color="pink")).add_to(m)   
m